# Import Libraries

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
load_dotenv()

# Set global variables 

In [ ]:
index_name: str = "demo"
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment="embeddings",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_SEARCH_KEY"),
    index_name=index_name,
    embedding_function=embeddings.embed_query
)

# Split PDF

In [ ]:
loader = PyPDFLoader("C:\\Users\\conne\\OneDrive\\Desktop\\GenAI_Demos\\RAG\\MicrosoftAzureHandbookk.pdf")
pages = loader.load_and_split()
print(pages)

# Upsert To Azure Search

In [ ]:
vector_store.add_documents(documents=pages)

# Query

In [ ]:
docs = vector_store.similarity_search(
    query="who was this guide written for?",
    k=3,
    search_type="hybrid"
)

# Add LLM Call

In [ ]:
llm = AzureChatOpenAI(
    azure_deployment="gpt4",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01"
)

In [ ]:
llm.invoke("answer the users original question: Who was this guide developed for? by looking at the below relevant documentation ")